# 노트북 2: Citation Crawler + Selector

**SNU AI Psychology Workshop - February 2026**

---

## 전체 흐름

```
┌─────────────────────────────────────────────────────────────┐
│  1. paperscraper    →   시드 후보 검색 (arXiv, PubMed)      │
│         ↓                                                   │
│  2. 시드 선택       →   시작점 지정 (번호 or DOI)           │
│         ↓                                                   │
│  3. Crawler         →   인용 네트워크 확장                  │
│      ├─ References  (시드가 인용한 논문)                    │
│      ├─ Citations   (시드를 인용한 논문)                    │
│      └─ Related     (인용 무관, 유사 논문)                  │
│         ↓                                                   │
│  4. Selector        →   쿼리 기반 관련성 필터링 (SPECTER2)  │
│         ↓                                                   │
│  5. 결과            →   Top 10 논문 추천                    │
└─────────────────────────────────────────────────────────────┘
```

**예시 주제: Climate Anxiety (기후 불안)**

| 추천 키워드 | 설명 |
|-------------|------|
| `"climate anxiety"` | 구문 검색 (정확히 일치) |
| `"eco-anxiety"` | 동의어 |
| `"climate change" AND "mental health"` | 복합 검색 |

---
# Part 1: Setup

**한 번만 실행하세요.** 모든 함수와 클래스가 정의됩니다.

예상 출력:
```
✓ 로컬 환경 / Colab 환경
✓ Semantic Scholar API Key 로드 성공
✓ search_with_paperscraper() 정의 완료
✓ SemanticScholarAPI 준비 완료
✓ CitationCrawler 정의 완료
✓ SPECTER2 로드 완료 (처음: ~500MB 다운로드)
```

In [ ]:
# Setup 1/6: 환경 설정
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("✓ Colab 환경")
except ImportError:
    if os.getcwd().endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("✓ 로컬 환경")

print(f"  작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Setup 2/6: 패키지 설치
if IN_COLAB:
    print("패키지 설치 중...")
    %pip install paperscraper requests pandas torch transformers adapters -q
    print("✓ 설치 완료")
else:
    print("로컬: 필요 시 아래 명령 실행")
    print("  pip install paperscraper requests pandas torch transformers adapters")

In [ ]:
# Setup 3/6: API Key & Imports
import requests
import time
import pandas as pd
from typing import List, Dict, Set, Optional
from collections import Counter

# API Key 로딩
SEMANTIC_SCHOLAR_API_KEY = None

if IN_COLAB:
    try:
        from google.colab import userdata
        SEMANTIC_SCHOLAR_API_KEY = userdata.get('SEMANTIC_SCHOLAR_API_KEY', None)
    except:
        pass
else:
    try:
        from dotenv import load_dotenv
        # 명시적으로 워크숍 루트의 .env 파일 경로 지정
        env_path = os.path.join(WORKSHOP_DIR, '.env')
        load_dotenv(env_path)
        SEMANTIC_SCHOLAR_API_KEY = os.getenv('SEMANTIC_SCHOLAR_API_KEY')
    except:
        pass

if SEMANTIC_SCHOLAR_API_KEY:
    print("✓ Semantic Scholar API Key 로드 성공")
    print(f"  Key: {SEMANTIC_SCHOLAR_API_KEY[:10]}...")
    print(f"  허용량: 5000 req/5분")
else:
    print("✗ Semantic Scholar API Key 없음")
    print("  허용량: 100 req/5분 (무료)")
    print("  발급: https://www.semanticscholar.org/product/api#api-key-form")

In [ ]:
# Setup 4/6: paperscraper 함수 + API 클래스들

def search_with_paperscraper(query: str, max_results: int = 5) -> pd.DataFrame:
    """paperscraper로 arXiv, PubMed 검색"""
    all_papers = []
    
    # arXiv
    try:
        from paperscraper.arxiv import get_arxiv_papers_api
        print("[arXiv] 검색 중...")
        df = get_arxiv_papers_api(query, max_results=max_results, verbose=False)
        for _, row in df.iterrows():
            all_papers.append({
                'source': 'arXiv', 'title': row.get('title', ''),
                'year': str(row.get('date', ''))[:4], 'doi': row.get('doi', ''),
                'abstract': str(row.get('abstract', ''))[:200] + '...'
            })
        print(f"  -> {len(df)}개")
    except Exception as e:
        print(f"[arXiv] 오류: {e}")
    
    # PubMed
    try:
        from paperscraper.pubmed import get_pubmed_papers
        print("[PubMed] 검색 중...")
        df = get_pubmed_papers(query, max_results=max_results)
        for _, row in df.iterrows():
            all_papers.append({
                'source': 'PubMed', 'title': row.get('title', ''),
                'year': str(row.get('date', ''))[:4], 'doi': row.get('doi', ''),
                'abstract': str(row.get('abstract', ''))[:200] + '...'
            })
        print(f"  -> {len(df)}개")
    except Exception as e:
        print(f"[PubMed] 오류: {e}")
    
    return pd.DataFrame(all_papers)


class OpenAlexAPI:
    """OpenAlex API 래퍼"""
    BASE_URL = "https://api.openalex.org"
    
    def __init__(self, email: str = None):
        self.params = {'mailto': email} if email else {}
    
    def _request(self, url: str, params: dict = None) -> dict:
        all_params = {**self.params, **(params or {})}
        try:
            response = requests.get(url, params=all_params, timeout=30)
            if response.status_code == 200:
                return response.json()
            return {}
        except:
            return {}
    
    def _extract_abstract(self, inv_idx: dict) -> str:
        """inverted index에서 abstract 복원"""
        if not inv_idx:
            return ''
        try:
            word_positions = [(word, pos) for word, positions in inv_idx.items() for pos in positions]
            word_positions.sort(key=lambda x: x[1])
            return ' '.join([wp[0] for wp in word_positions[:200]])
        except:
            return ''
    
    def get_paper_by_doi(self, doi: str) -> Dict:
        """DOI로 논문 정보 조회"""
        result = self._request(f"{self.BASE_URL}/works/doi:{doi}")
        if not result:
            return {}
        return {
            'openalex_id': result.get('id', ''),
            'title': result.get('title', ''),
            'year': result.get('publication_year'),
            'cited_by_count': result.get('cited_by_count', 0),
            'abstract': self._extract_abstract(result.get('abstract_inverted_index')),
            'referenced_works': result.get('referenced_works', []),
        }
    
    def get_references(self, doi: str, limit: int = 50) -> List[Dict]:
        """논문의 references 조회 (cited_by 엔드포인트 사용)"""
        paper = self.get_paper_by_doi(doi)
        if not paper or not paper.get('referenced_works'):
            return []
        
        ref_ids = paper['referenced_works'][:limit]
        if not ref_ids:
            return []
        
        # 배치로 조회 (한 번의 API 호출로 최대 50개)
        short_ids = [oid.replace('https://openalex.org/', '') for oid in ref_ids if isinstance(oid, str)]
        filter_str = "|".join(short_ids)
        
        params = {
            'filter': f'openalex:{filter_str}',
            'per_page': limit,
            'select': 'id,title,publication_year,cited_by_count,abstract_inverted_index'
        }
        
        result = self._request(f"{self.BASE_URL}/works", params)
        if not result or 'results' not in result:
            return []
        
        papers = []
        for r in result.get('results', []):
            papers.append({
                'openalex_id': r.get('id', ''),
                'paperId': r.get('id', '').replace('https://openalex.org/', ''),
                'title': r.get('title', ''),
                'year': r.get('publication_year'),
                'citationCount': r.get('cited_by_count', 0),
                'abstract': self._extract_abstract(r.get('abstract_inverted_index'))
            })
        
        return papers


class SemanticScholarAPI:
    """Semantic Scholar API 래퍼"""
    BASE_URL = "https://api.semanticscholar.org/graph/v1"
    RECO_URL = "https://api.semanticscholar.org/recommendations/v1"
    
    def __init__(self, api_key: str = None):
        self.headers = {'x-api-key': api_key} if api_key else {}
        self.publisher_blocked = False
    
    def _request(self, url: str, params: dict = None) -> dict:
        for attempt in range(3):
            try:
                response = requests.get(url, params=params, headers=self.headers, timeout=30)
                if response.status_code == 200:
                    return response.json()
                elif response.status_code == 429:
                    wait_time = 5 * (attempt + 1)
                    print(f"  Rate limit - {wait_time}초 대기...")
                    time.sleep(wait_time)
                else:
                    return {}
            except:
                return {}
        return {}
    
    def get_paper(self, doi: str = None, title: str = None) -> Dict:
        self.publisher_blocked = False
        params = {"fields": "paperId,title,abstract,year,citationCount,authors,externalIds"}
        if doi:
            self.current_doi = doi
            if doi.startswith("10.48550/arXiv."):
                paper_id = f"arXiv:{doi.replace('10.48550/arXiv.', '')}"
            else:
                paper_id = f"DOI:{doi}"
            result = self._request(f"{self.BASE_URL}/paper/{paper_id}", params)
        elif title:
            self.current_doi = None
            params.update({"query": title, "limit": 1})
            result = self._request(f"{self.BASE_URL}/paper/search", params)
            result = (result.get('data') or [{}])[0] if result else {}
        else:
            return {}
        
        if result and result.get('externalIds'):
            self.current_doi = result['externalIds'].get('DOI')
        
        return result if result else {}
    
    def get_references(self, paper_id: str, limit: int = 50) -> List[Dict]:
        params = {"fields": "paperId,title,abstract,year,citationCount", "limit": limit}
        result = self._request(f"{self.BASE_URL}/paper/{paper_id}/references", params)
        data = (result.get('data') or []) if result else []
        refs = [i['citedPaper'] for i in data if i.get('citedPaper', {}).get('paperId')]
        
        # 출판사 차단 감지
        if not refs and result:
            self.publisher_blocked = True
        
        return refs
    
    def get_citations(self, paper_id: str, limit: int = 50) -> List[Dict]:
        params = {"fields": "paperId,title,abstract,year,citationCount", "limit": limit}
        result = self._request(f"{self.BASE_URL}/paper/{paper_id}/citations", params)
        data = (result.get('data') or []) if result else []
        return [i['citingPaper'] for i in data if i.get('citingPaper', {}).get('paperId')]
    
    def get_related(self, paper_id: str, limit: int = 20) -> List[Dict]:
        params = {"fields": "paperId,title,abstract,year,citationCount", "limit": limit}
        result = self._request(f"{self.RECO_URL}/papers/forpaper/{paper_id}", params)
        return (result.get('recommendedPapers') or []) if result else []


# API 인스턴스 생성
api = SemanticScholarAPI(api_key=SEMANTIC_SCHOLAR_API_KEY)
openalex = OpenAlexAPI()

print("✓ search_with_paperscraper() 정의 완료")
print("✓ SemanticScholarAPI 준비 완료")
print("✓ OpenAlexAPI 준비 완료")

In [ ]:
# Setup 5/6: CitationCrawler 클래스

class CitationCrawler:
    """인용 네트워크 + 관련 논문 탐색 (Semantic Scholar + OpenAlex)"""
    
    def __init__(self, api: SemanticScholarAPI, openalex_api: OpenAlexAPI = None):
        self.api = api
        self.openalex = openalex_api
        self.visited: Set[str] = set()
        self.papers: Dict[str, Dict] = {}
    
    def crawl(self, seed_paper: Dict, seed_doi: str = None, refs_limit: int = 15, 
              cites_limit: int = 15, related_limit: int = 10, max_total: int = 50) -> List[Dict]:
        self.visited.clear()
        self.papers.clear()
        
        paper_id = seed_paper.get('paperId')
        if not paper_id:
            print("오류: paperId 없음")
            return []
        
        self.visited.add(paper_id)
        seed_paper['source'] = 'seed'
        self.papers[paper_id] = seed_paper
        print(f"시드: {seed_paper.get('title', 'N/A')[:50]}...")
        
        # References (Semantic Scholar)
        print("\n[References] 수집 중...")
        refs = self.api.get_references(paper_id, limit=refs_limit)
        
        # Semantic Scholar 실패 시 OpenAlex 시도
        if not refs and self.openalex and seed_doi:
            print("  → OpenAlex에서 시도...")
            refs = self.openalex.get_references(seed_doi, limit=refs_limit)
        
        for p in refs:
            pid = p.get('paperId') or p.get('openalex_id', '')
            if pid and pid not in self.visited and len(self.papers) < max_total:
                self.visited.add(pid)
                p['source'] = 'reference'
                self.papers[pid] = p
        print(f"  -> {len([p for p in self.papers.values() if p.get('source') == 'reference'])}개")
        time.sleep(0.3)
        
        # Citations (Semantic Scholar만)
        print("[Citations] 수집 중...")
        for p in self.api.get_citations(paper_id, limit=cites_limit):
            pid = p.get('paperId')
            if pid and pid not in self.visited and len(self.papers) < max_total:
                self.visited.add(pid)
                p['source'] = 'citation'
                self.papers[pid] = p
        print(f"  -> {len([p for p in self.papers.values() if p.get('source') == 'citation'])}개")
        time.sleep(0.3)
        
        # Related (Semantic Scholar만)
        print("[Related] 수집 중...")
        for p in self.api.get_related(paper_id, limit=related_limit):
            pid = p.get('paperId')
            if pid and pid not in self.visited and len(self.papers) < max_total:
                self.visited.add(pid)
                p['source'] = 'related'
                self.papers[pid] = p
        print(f"  -> {len([p for p in self.papers.values() if p.get('source') == 'related'])}개")
        
        print(f"\n총 {len(self.papers)}개 수집 완료")
        return list(self.papers.values())

print("✓ CitationCrawler 정의 완료")

In [ ]:
# Setup 6/6: SPECTER2Selector (모델 다운로드 ~500MB)

import torch
from torch.nn.functional import cosine_similarity

class SPECTER2Selector:
    def __init__(self, device='cpu'):
        print("SPECTER2 로딩 중... (처음: ~500MB 다운로드)")
        from transformers import AutoTokenizer
        from adapters import AutoAdapterModel
        
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
        self.model = AutoAdapterModel.from_pretrained('allenai/specter2_base')
        self.model.load_adapter("allenai/specter2", source="hf", load_as="proximity", set_active=True)
        self.model.to(device).eval()
        self.device = device
        print(f"✓ SPECTER2 로드 완료")
    
    def get_embedding(self, title: str, abstract: str = "") -> torch.Tensor:
        text = title + self.tokenizer.sep_token + (abstract or "")
        inputs = self.tokenizer([text], padding=True, truncation=True, return_tensors="pt", max_length=512).to(self.device)
        with torch.no_grad():
            return self.model(**inputs).last_hidden_state[:, 0, :]
    
    def select_top_papers(self, query: str, papers: List[Dict], top_k: int = 10) -> List[Dict]:
        if not papers:
            return []
        print(f"\n{len(papers)}개 논문 평가 중...")
        query_emb = self.get_embedding(query)
        for i, paper in enumerate(papers):
            if (i + 1) % 10 == 0:
                print(f"  {i + 1}/{len(papers)}")
            paper_emb = self.get_embedding(paper.get('title', ''), paper.get('abstract', ''))
            paper['relevance_score'] = round(cosine_similarity(query_emb, paper_emb).item(), 4)
        papers.sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        print(f"✓ 상위 {top_k}개 선택 완료")
        return papers[:top_k]

# 인스턴스 생성 (모델 로드)
selector = SPECTER2Selector(device='cpu')

---
# Part 2: 실행

Setup 완료 후, 아래 Step들을 순서대로 실행하세요.

## Step 1: 키워드 검색 (paperscraper)

### paperscraper란?
[paperscraper](https://github.com/jannisborn/paperscraper)는 여러 학술 데이터베이스를 통합 검색하는 Python 라이브러리입니다.

**지원 데이터베이스:**
| DB | 분야 | 특징 |
|---|---|---|
| **arXiv** | CS, 물리, 수학, 통계, AI/ML | 프리프린트 중심, 최신 연구 빠름 |
| **PubMed** | 의학, 생명과학, 심리학 | 피어리뷰 저널 중심, 임상 연구 |
| bioRxiv | 생명과학 프리프린트 | (별도 dump 필요) |
| medRxiv | 의학 프리프린트 | (별도 dump 필요) |

> 💡 **이 노트북에서는 arXiv + PubMed를 사용합니다.**

**왜 Google Scholar/Scopus가 아닌가?**
- Google Scholar: 공식 API 없음 (스크래핑 필요 → 차단 위험)
- Scopus: 유료 API (기관 구독 필요)
- Semantic Scholar: **무료 공식 API** 제공 → Crawler에서 사용

---

### 쿼리 형식
- 단순: `climate anxiety`
- AND: `climate AND anxiety`
- OR: `climate OR weather`
- 구문: `"eco-anxiety"`

In [ ]:
# Step 1: 키워드 검색
QUERY = '"climate anxiety"'  # <- 수정 가능! (따옴표로 구문 검색)

print(f"쿼리: {QUERY}")
print("=" * 50)
candidates_df = search_with_paperscraper(QUERY, max_results=5)
print(f"\n총 {len(candidates_df)}개 후보 발견")

In [ ]:
# 후보 목록 보기
from IPython.display import display, HTML

if len(candidates_df) > 0:
    candidates_df.insert(0, 'No', range(1, len(candidates_df) + 1))
    print("\n후보 논문 목록:")
    print("=" * 70)
    
    html_rows = []
    for _, row in candidates_df.iterrows():
        title = row['title'][:55] + '...' if len(str(row['title'])) > 55 else row['title']
        doi = row['doi']
        doi_link = f'<a href="https://doi.org/{doi}" target="_blank">{doi}</a>' if doi else "N/A"
        
        html_rows.append(f"""
        <tr>
            <td style="vertical-align:top; padding:5px;"><b>{row['No']}</b></td>
            <td style="vertical-align:top; padding:5px;">[{row['source']}]</td>
            <td style="padding:5px;">
                {title}<br>
                <small>연도: {row['year']} | DOI: {doi_link}</small>
            </td>
        </tr>
        """)
    
    html_table = f"""
    <table style="border-collapse:collapse; width:100%;">
        {''.join(html_rows)}
    </table>
    """
    display(HTML(html_table))
else:
    print("검색 결과 없음. 쿼리를 변경하세요.")

## Step 2: 시드 선택

위 목록에서 **번호**로 선택하거나, **DOI를 직접 입력**하세요.

In [ ]:
# Step 2: 시드 선택
SEED_INDEX = 1        # <- 번호 (1부터 시작)
SEED_DOI = None       # <- DOI 입력 시 번호 무시

# 시드 논문 가져오기
if SEED_DOI:
    print(f"DOI로 검색: {SEED_DOI}")
    seed_paper = api.get_paper(doi=SEED_DOI)
elif len(candidates_df) >= SEED_INDEX:
    row = candidates_df.iloc[SEED_INDEX - 1]
    print(f"{SEED_INDEX}번 논문 검색 중...")
    seed_paper = api.get_paper(doi=row['doi']) if row.get('doi') else api.get_paper(title=row['title'])
else:
    print(f"오류: {SEED_INDEX}번 논문 없음")
    seed_paper = {}

# 결과 출력
if seed_paper.get('paperId'):
    print("\n" + "=" * 60)
    print("✓ 시드 논문 선택 완료")
    print("=" * 60)
    print(f"제목: {seed_paper.get('title', 'N/A')}")
    print(f"연도: {seed_paper.get('year', 'N/A')} | 인용: {seed_paper.get('citationCount', 0)}")
    authors = seed_paper.get('authors', [])
    if authors:
        print(f"저자: {', '.join([a.get('name','') for a in authors[:3]])}" + (" 외" if len(authors) > 3 else ""))
else:
    print("\n✗ 시드 논문을 찾을 수 없습니다.")

## Step 3: Crawler 실행

### Citation Crawler란?
시드 논문에서 출발하여 **3가지 방향**으로 관련 논문을 수집합니다.

| 방향 | 설명 | API |
|------|------|-----|
| **References** | 시드가 인용한 논문 (과거 연구) | Semantic Scholar → OpenAlex |
| **Citations** | 시드를 인용한 논문 (후속 연구) | Semantic Scholar |
| **Related** | 인용 관계 없이 유사한 논문 | Semantic Scholar |

```
              ┌─────────────┐
    과거 ←── │  Seed Paper │ ──→ 최신
  References └──────┬──────┘  Citations
                    │
                    ▼
                 Related
              (인용 무관 유사)
```

> 💡 **Related가 중요한 이유**: 서로 인용하지 않지만 밀접한 관련이 있는 논문을 놓치지 않습니다.

---

### 사용 API

| API | 설명 | 특징 |
|-----|------|------|
| **[Semantic Scholar](https://api.semanticscholar.org/)** | Allen AI의 학술 검색 API | 추천/관련 논문 기능 강점 |
| **[OpenAlex](https://openalex.org/)** | 완전 무료 오픈 학술 DB | 출판사 제한 없음, References 강점 |

> ⚠️ 일부 출판사는 Semantic Scholar에 데이터를 제공하지 않습니다. 이 경우 **OpenAlex**에서 References를 가져옵니다.

In [ ]:
# Step 3: Crawler 실행
if seed_paper.get('paperId'):
    crawler = CitationCrawler(api, openalex_api=openalex)
    
    # 시드 DOI 가져오기 (OpenAlex fallback용)
    seed_doi = None
    if hasattr(api, 'current_doi'):
        seed_doi = api.current_doi
    elif 'candidates_df' in dir() and SEED_INDEX <= len(candidates_df):
        seed_doi = candidates_df.iloc[SEED_INDEX - 1].get('doi')
    
    crawled_papers = crawler.crawl(
        seed_paper=seed_paper,
        seed_doi=seed_doi,
        refs_limit=15, cites_limit=15, related_limit=10, max_total=50
    )
    
    # 결과 검증
    ref_count = len([p for p in crawled_papers if p.get('source') == 'reference'])
    cite_count = len([p for p in crawled_papers if p.get('source') == 'citation'])
    
    if ref_count == 0:
        print("\n⚠️  References = 0: 비정상입니다.")
    if cite_count == 0 and seed_paper.get('year', 2020) < 2024:
        print("   (오래된 논문인데 Citations = 0이면 문제일 수 있습니다)")
else:
    print("시드 논문이 없습니다. Step 2를 확인하세요.")
    crawled_papers = []

In [ ]:
# 크롤링 통계
if crawled_papers:
    counts = Counter(p.get('source', '?') for p in crawled_papers)
    print("=" * 40)
    print("크롤링 결과 요약")
    print("=" * 40)
    for src, cnt in counts.items():
        icon = {'seed':'🌱', 'reference':'📚', 'citation':'📝', 'related':'🔗'}.get(src, '📄')
        print(f"  {icon} {src}: {cnt}개")

## Step 4: Selector 실행

### SPECTER2란?
[SPECTER2](https://github.com/allenai/SPECTER2)는 Allen AI에서 개발한 **학술 논문 임베딩 모델**입니다.

- **기반**: SciBERT (과학 문헌 특화 BERT)
- **학습 데이터**: 700만+ 논문의 인용 관계
- **특징**: 제목+초록만으로 논문의 의미를 768차원 벡터로 표현

**원리:**
```
┌─────────────┐     ┌─────────────┐
│   Query     │     │   Paper     │
│ "climate    │     │ title +     │
│  anxiety"   │     │ abstract    │
└──────┬──────┘     └──────┬──────┘
       │                   │
       ▼                   ▼
   SPECTER2            SPECTER2
       │                   │
       ▼                   ▼
┌─────────────┐     ┌─────────────┐
│  Query      │     │  Paper      │
│  Embedding  │     │  Embedding  │
│  (768-dim)  │     │  (768-dim)  │
└──────┬──────┘     └──────┬──────┘
       │                   │
       └───────┬───────────┘
               ▼
        Cosine Similarity
         (0.0 ~ 1.0)
               │
               ▼
        Relevance Score
```

> 💡 **Selector의 역할**: Crawler가 수집한 논문들 중에서 **쿼리와 의미적으로 가장 유사한** 논문을 선별합니다.

> ⚠️ **쿼리가 바뀌면 결과가 달라집니다**: 같은 논문 풀이라도 쿼리가 다르면 Query Embedding이 달라지고, 각 논문과의 유사도 점수가 바뀝니다. → **실험 B**에서 직접 확인해보세요!

**참고 자료:**
- 논문: [SciRepEval: A Multi-Format Benchmark for Scientific Document Representations](https://arxiv.org/abs/2211.13308)
- GitHub: [allenai/SPECTER2](https://github.com/allenai/SPECTER2)
- HuggingFace: [allenai/specter2_base](https://huggingface.co/allenai/specter2_base)

In [ ]:
# Step 4: Selector 실행
if crawled_papers:
    selected_papers = selector.select_top_papers(
        query=QUERY,
        papers=crawled_papers.copy(),
        top_k=10
    )
else:
    print("크롤링 결과 없음. Step 3을 확인하세요.")
    selected_papers = []

## Step 5: 최종 결과

In [ ]:
# Step 5: 최종 결과
from IPython.display import display, HTML

if selected_papers:
    print("=" * 70)
    print(f"Top 10 관련 논문")
    print(f"쿼리: {QUERY}")
    print("=" * 70)
    
    html_rows = []
    for i, p in enumerate(selected_papers, 1):
        icon = {'seed':'🌱', 'reference':'📚', 'citation':'📝', 'related':'🔗'}.get(p.get('source',''), '📄')
        title = p.get('title', 'N/A')
        score = p.get('relevance_score', 0)
        year = p.get('year', '?')
        citations = p.get('citationCount', 0)
        source = p.get('source', '')
        
        # DOI 또는 Semantic Scholar 링크
        paper_id = p.get('paperId', '')
        ext_ids = p.get('externalIds', {}) or {}
        doi = ext_ids.get('DOI', '') if isinstance(ext_ids, dict) else ''
        
        if doi:
            link = f'<a href="https://doi.org/{doi}" target="_blank">{doi}</a>'
        elif paper_id:
            link = f'<a href="https://www.semanticscholar.org/paper/{paper_id}" target="_blank">SS:{paper_id[:12]}...</a>'
        else:
            link = "N/A"
        
        html_rows.append(f"""
        <tr>
            <td style="padding:5px;"><b>{i}</b></td>
            <td style="padding:5px;">[{score:.3f}]</td>
            <td style="padding:5px;">{icon}</td>
            <td style="padding:5px;">
                {title[:60]}{'...' if len(title) > 60 else ''}<br>
                <small>연도: {year} | 인용: {citations} | {source} | {link}</small>
            </td>
        </tr>
        """)
    
    html_table = f"""
    <table style="border-collapse:collapse; width:100%;">
        {''.join(html_rows)}
    </table>
    <p style="color:#666; font-size:12px;">💡 DOI를 복사해서 Part 3 실험에 시드로 사용할 수 있습니다.</p>
    """
    display(HTML(html_table))
else:
    print("선택된 논문이 없습니다. Step 4를 확인하세요.")

---
# Part 3: 비교 실험

**두 가지 실험으로 검색 결과에 영향을 미치는 요소를 탐구합니다.**

| 실험 | 고정 | 변경 | 질문 |
|------|------|------|------|
| **실험 A** | 쿼리 | 시드 | 시드가 다르면 결과가 얼마나 달라질까? |
| **실험 B** | 시드 | 쿼리 | 쿼리가 다르면 결과가 얼마나 달라질까? |

---

## 실험 A: 시드 비교 (같은 쿼리, 다른 시드)

### 시드 선택 방법

**Step 1 (cell-11)에서 검색된 논문 목록**을 참고하여 2개의 시드를 선택하세요.

```
예: Step 1 결과가 다음과 같다면
   1. [PubMed] Climate change and mental health... (DOI: 10.1016/j.ajp.2026.104859)
   2. [PubMed] The generational happiness switch... (DOI: 10.3389/fpsyg.2025.1706473)
   3. [arXiv] Climate anxiety in adolescents...   (DOI: 10.48550/arXiv.2401.12345)
   
   → SEED_1_DOI = "10.1016/j.ajp.2026.104859"  (1번 선택)
   → SEED_2_DOI = "10.3389/fpsyg.2025.1706473" (2번 선택)
```

> 💡 **팁**: 서로 다른 관점의 논문을 선택하면 비교가 더 흥미로워집니다.  
> 예: 임상 연구 vs 이론 연구, 청소년 대상 vs 성인 대상

In [ ]:
# 파이프라인 함수
def run_pipeline(seed_doi=None, seed_title=None, query="climate anxiety", max_papers=30, top_k=10):
    result = {'seed': None, 'crawled': [], 'selected': []}
    
    seed = api.get_paper(doi=seed_doi, title=seed_title)
    if not seed.get('paperId'):
        print("✗ 시드 논문을 찾을 수 없습니다.")
        return result
    result['seed'] = seed
    print(f"시드: {seed.get('title', 'N/A')[:50]}...")
    
    crawler = CitationCrawler(api)
    result['crawled'] = crawler.crawl(seed, refs_limit=10, cites_limit=10, related_limit=5, max_total=max_papers)
    result['selected'] = selector.select_top_papers(query, result['crawled'].copy(), top_k)
    return result

print("✓ run_pipeline() 정의 완료")

In [ ]:
# 실험 A: 시드 비교 실행
# ============================================================
# Step 1 (cell-11) 결과에서 DOI 2개를 복사해서 붙여넣으세요!
# ============================================================

SEED_1_DOI = ""  # <- cell-11에서 1번 논문의 DOI
SEED_2_DOI = ""  # <- cell-11에서 2번 논문의 DOI

COMPARE_QUERY = '"climate anxiety"'

# ============================================================
# 입력 검증
# ============================================================
if not COMPARE_QUERY or COMPARE_QUERY.strip() == "":
    print("⚠️  경고: COMPARE_QUERY가 비어있습니다!")
    print("   비교에 사용할 쿼리를 입력하세요.")
    print('   예: COMPARE_QUERY = \'"climate anxiety"\'')
    result_1, result_2 = {'selected': []}, {'selected': []}
elif not SEED_1_DOI or not SEED_2_DOI:
    print("⚠️  경고: 시드 DOI가 비어있습니다!")
    print("   Step 1 (cell-11) 결과에서 DOI 2개를 복사하세요.")
    print('   예: SEED_1_DOI = "10.1016/j.ajp.2026.104859"')
    result_1, result_2 = {'selected': []}, {'selected': []}
else:
    print(f"쿼리: {COMPARE_QUERY}")
    print("=" * 60)
    print("1️⃣ 시드 1 실행")
    print("=" * 60)
    result_1 = run_pipeline(seed_doi=SEED_1_DOI, query=COMPARE_QUERY)

    print("\n" + "=" * 60)
    print("2️⃣ 시드 2 실행")
    print("=" * 60)
    result_2 = run_pipeline(seed_doi=SEED_2_DOI, query=COMPARE_QUERY)

In [ ]:
# 실험 A: 결과 비교 테이블
import pandas as pd
from IPython.display import display, HTML

if result_1.get('selected') and result_2.get('selected'):
    # 시드 정보
    seed_1_title = result_1['seed'].get('title', 'N/A')[:40] + "..."
    seed_2_title = result_2['seed'].get('title', 'N/A')[:40] + "..."
    
    # Top 10 비교 데이터프레임 생성
    def make_result_df(result, label):
        rows = []
        for i, p in enumerate(result['selected'], 1):
            rows.append({
                '순위': i,
                '점수': f"{p.get('relevance_score', 0):.3f}",
                '제목': p.get('title', 'N/A')[:45] + "...",
                '연도': p.get('year', '?'),
                '인용': p.get('citationCount', 0),
                '출처': p.get('source', ''),
                'paperId': p.get('paperId', '')
            })
        return pd.DataFrame(rows)
    
    df_1 = make_result_df(result_1, '1')
    df_2 = make_result_df(result_2, '2')
    
    # 공통 논문 찾기
    ids_1 = set(df_1['paperId'])
    ids_2 = set(df_2['paperId'])
    common_ids = ids_1 & ids_2
    
    # 결과 출력
    print("=" * 70)
    print("📊 실험 A: 시드 비교 결과")
    print("=" * 70)
    print(f"\n1️⃣ 시드 1: {seed_1_title}")
    print(f"2️⃣ 시드 2: {seed_2_title}")
    print(f"\n🔗 공통 논문: {len(common_ids)}개 / 10개")
    
    # 비교 테이블 (나란히)
    print("\n" + "=" * 70)
    print("1️⃣ 시드 1 결과                      2️⃣ 시드 2 결과")
    print("=" * 70)
    
    comparison_rows = []
    for i in range(10):
        s1_title = df_1.iloc[i]['제목'][:30] if i < len(df_1) else ""
        s2_title = df_2.iloc[i]['제목'][:30] if i < len(df_2) else ""
        comparison_rows.append({
            '#': i+1,
            '시드1 점수': df_1.iloc[i]['점수'],
            '시드1 논문': s1_title,
            '': '│',
            '시드2 점수': df_2.iloc[i]['점수'],
            '시드2 논문': s2_title,
            '공통': "⭐" if df_1.iloc[i]['paperId'] in common_ids or df_2.iloc[i]['paperId'] in common_ids else ""
        })
    
    comparison_df = pd.DataFrame(comparison_rows)
    display(comparison_df.style.hide(axis='index'))
    
    # 공통 논문 상세
    if common_ids:
        print("\n" + "=" * 70)
        print("⭐ 공통으로 선택된 논문")
        print("=" * 70)
        for p in result_1['selected']:
            if p.get('paperId') in common_ids:
                print(f"  • {p.get('title', 'N/A')[:60]}...")
                print(f"    연도: {p.get('year', '?')} | 인용: {p.get('citationCount', 0)}")
    else:
        print("\n⚠️  공통 논문 없음! 시드 선택이 결과에 큰 영향을 미칩니다.")
    
    # 결과 저장용 변수
    comparison_result = {
        'query': COMPARE_QUERY,
        'seed_1': result_1['seed'],
        'seed_2': result_2['seed'],
        'top10_1': result_1['selected'],
        'top10_2': result_2['selected'],
        'common_count': len(common_ids),
        'common_ids': list(common_ids)
    }
else:
    print("⚠️  비교할 결과가 없습니다. 위 셀을 먼저 실행하세요.")

---

## 실험 B: 쿼리 비교 (같은 시드, 다른 쿼리)

**같은 논문 풀에서 쿼리만 바꾸면 Top 10이 어떻게 달라질까요?**

예시:
- 쿼리 1: `"climate anxiety"` → 불안 중심
- 쿼리 2: `"eco-paralysis"` → 무력감/행동 억제 중심

In [ ]:
# 실험 B: 쿼리 비교 실행
# ============================================================
# 같은 시드에서 쿼리만 바꾸면 결과가 어떻게 달라질까?
# ============================================================

FIXED_SEED_DOI = ""  # <- Step 1 (cell-11) 또는 Step 5 (cell-20) 결과에서 DOI 복사

QUERY_1 = '"climate anxiety"'    # 불안 중심
QUERY_2 = '"eco-paralysis"'      # 무력감/행동 억제 중심

# ============================================================
# 입력 검증
# ============================================================
if not FIXED_SEED_DOI:
    print("⚠️  경고: FIXED_SEED_DOI가 비어있습니다!")
    print("   Step 1 (cell-11) 또는 Step 5 (cell-20) 결과에서 DOI를 복사하세요.")
    print('   예: FIXED_SEED_DOI = "10.1016/S2542-5196(21)00278-3"')
    query_result_1, query_result_2 = {'selected': [], 'crawled': []}, {'selected': [], 'crawled': []}
elif not QUERY_1 or not QUERY_2:
    print("⚠️  경고: 쿼리가 비어있습니다!")
    print("   QUERY_1과 QUERY_2를 모두 입력하세요.")
    query_result_1, query_result_2 = {'selected': [], 'crawled': []}, {'selected': [], 'crawled': []}
else:
    print(f"🔒 고정 시드: {FIXED_SEED_DOI}")
    print(f"📝 쿼리 1: {QUERY_1}")
    print(f"📝 쿼리 2: {QUERY_2}")
    print("=" * 60)
    
    # 시드 논문 가져오기 (한 번만)
    print("시드 논문 로드 중...")
    fixed_seed = api.get_paper(doi=FIXED_SEED_DOI)
    
    if not fixed_seed.get('paperId'):
        print("✗ 시드 논문을 찾을 수 없습니다.")
        query_result_1, query_result_2 = {'selected': [], 'crawled': []}, {'selected': [], 'crawled': []}
    else:
        print(f"✓ 시드: {fixed_seed.get('title', 'N/A')[:50]}...")
        
        # Crawler 1회만 실행 (논문 풀은 동일)
        print("\n[Crawler] 논문 풀 수집 중...")
        crawler = CitationCrawler(api)
        shared_papers = crawler.crawl(fixed_seed, refs_limit=15, cites_limit=15, related_limit=10, max_total=50)
        
        # Selector: 쿼리 1
        print("\n" + "=" * 60)
        print(f"🔍 쿼리 1: {QUERY_1}")
        print("=" * 60)
        selected_1 = selector.select_top_papers(QUERY_1, [p.copy() for p in shared_papers], top_k=10)
        query_result_1 = {'seed': fixed_seed, 'crawled': shared_papers, 'selected': selected_1}
        
        # Selector: 쿼리 2
        print("\n" + "=" * 60)
        print(f"🔍 쿼리 2: {QUERY_2}")
        print("=" * 60)
        selected_2 = selector.select_top_papers(QUERY_2, [p.copy() for p in shared_papers], top_k=10)
        query_result_2 = {'seed': fixed_seed, 'crawled': shared_papers, 'selected': selected_2}

In [ ]:
# 실험 B: 쿼리 비교 결과 테이블
if query_result_1.get('selected') and query_result_2.get('selected'):
    # Top 10 비교 데이터프레임 생성
    def make_query_result_df(selected):
        rows = []
        for i, p in enumerate(selected, 1):
            rows.append({
                '순위': i,
                '점수': f"{p.get('relevance_score', 0):.3f}",
                '제목': p.get('title', 'N/A')[:45] + "...",
                '연도': p.get('year', '?'),
                '인용': p.get('citationCount', 0),
                'paperId': p.get('paperId', '')
            })
        return pd.DataFrame(rows)
    
    qdf_1 = make_query_result_df(query_result_1['selected'])
    qdf_2 = make_query_result_df(query_result_2['selected'])
    
    # 공통 논문 찾기
    qids_1 = set(qdf_1['paperId'])
    qids_2 = set(qdf_2['paperId'])
    q_common_ids = qids_1 & qids_2
    
    # 결과 출력
    print("=" * 70)
    print("📊 쿼리 비교 결과")
    print("=" * 70)
    print(f"\n🔒 고정 시드: {query_result_1['seed'].get('title', 'N/A')[:50]}...")
    print(f"\n📝 쿼리 1: {QUERY_1}")
    print(f"📝 쿼리 2: {QUERY_2}")
    print(f"\n🔗 공통 논문: {len(q_common_ids)}개 / 10개")
    print(f"📊 논문 풀 크기: {len(query_result_1['crawled'])}개 (Crawler 결과)")
    
    # 비교 테이블 (나란히)
    print("\n" + "=" * 70)
    print(f"🔍 쿼리 1 결과                      🔍 쿼리 2 결과")
    print("=" * 70)
    
    q_comparison_rows = []
    for i in range(10):
        q1_title = qdf_1.iloc[i]['제목'][:28] if i < len(qdf_1) else ""
        q2_title = qdf_2.iloc[i]['제목'][:28] if i < len(qdf_2) else ""
        q_comparison_rows.append({
            '#': i+1,
            'Q1 점수': qdf_1.iloc[i]['점수'] if i < len(qdf_1) else "",
            'Q1 논문': q1_title,
            '': '│',
            'Q2 점수': qdf_2.iloc[i]['점수'] if i < len(qdf_2) else "",
            'Q2 논문': q2_title,
            '공통': "⭐" if (i < len(qdf_1) and qdf_1.iloc[i]['paperId'] in q_common_ids) else ""
        })
    
    q_comparison_df = pd.DataFrame(q_comparison_rows)
    display(q_comparison_df.style.hide(axis='index'))
    
    # 인사이트
    print("\n" + "=" * 70)
    print("💡 인사이트")
    print("=" * 70)
    
    if len(q_common_ids) >= 7:
        print("  → 두 쿼리가 매우 유사한 결과를 가져옵니다 (공통 70%+)")
        print("  → 쿼리 간 의미적 차이가 작을 수 있습니다.")
    elif len(q_common_ids) >= 4:
        print("  → 두 쿼리가 부분적으로 중복됩니다 (공통 40-60%)")
        print("  → 각 쿼리가 다른 관점의 논문을 발견합니다.")
    else:
        print("  → 두 쿼리가 상당히 다른 결과를 가져옵니다 (공통 30% 미만)")
        print("  → 쿼리 선택이 Selector 결과에 큰 영향을 미칩니다!")
    
    # 점수 차이 분석
    avg_score_1 = sum(p.get('relevance_score', 0) for p in query_result_1['selected']) / 10
    avg_score_2 = sum(p.get('relevance_score', 0) for p in query_result_2['selected']) / 10
    print(f"\n  📈 평균 관련성 점수:")
    print(f"     쿼리 1 ({QUERY_1}): {avg_score_1:.3f}")
    print(f"     쿼리 2 ({QUERY_2}): {avg_score_2:.3f}")
    
    if abs(avg_score_1 - avg_score_2) > 0.05:
        higher = QUERY_1 if avg_score_1 > avg_score_2 else QUERY_2
        print(f"     → '{higher}' 쿼리가 수집된 논문 풀과 더 잘 맞습니다.")
else:
    print("⚠️  비교할 결과가 없습니다. 위 셀을 먼저 실행하세요.")

---

# Part 4: 결과 공유 & 토론

## 🔗 결과 제출 방법

### 참가자 워크플로우
1. **실험 A와 B 모두 실행** (위 셀들)
2. 아래 셀에서 **이름** 입력
3. 셀 실행 → **파란 버튼 클릭**
4. 폼에서 **토론 질문 4개 작성** 후 제출

### 📎 결과 확인
- **[공유 시트 보기](https://docs.google.com/spreadsheets/d/15jyTrqGY7Po5iLcXFrv_kwyUNkCC9YMX6kypPMs-bAc/edit?usp=sharing)** - 모든 참가자 결과
- **[직접 제출 (폼)](https://forms.gle/dYNbvMeeBMqxSmLa7)** - 수동 입력용

---

### 💡 토론 질문 (폼에서 작성)
1. **시드 선택의 영향**: 같은 쿼리여도 시드에 따라 결과가 얼마나 달라졌나요?
2. **쿼리 선택의 영향**: 같은 논문 풀에서 쿼리에 따라 Top 10이 얼마나 바뀌었나요?
3. **공통 논문의 특징**: 어떤 논문이 공통으로 선택되었나요? 왜 그럴까요?
4. **내 연구에 적용한다면**: 어떤 시드와 쿼리 조합이 효과적일까요?

In [ ]:
# 결과 제출
# ============================================================
# 이름만 입력하세요! 나머지는 자동으로 채워집니다.
# ============================================================

PARTICIPANT_NAME = ""  # <- 본인 이름

# ============================================================
# 실험 결과 자동 수집
# ============================================================
import urllib.parse
from datetime import datetime
from IPython.display import display, HTML

def format_top10(selected_papers):
    """Top 10 논문을 텍스트로 포맷팅"""
    if not selected_papers:
        return "(결과 없음)"
    lines = []
    for i, p in enumerate(selected_papers[:10], 1):
        title = p.get('title', 'N/A')[:50]
        score = p.get('relevance_score', 0)
        lines.append(f"{i}. [{score:.3f}] {title}...")
    return "\n".join(lines)

def format_common_papers(papers_a, papers_b):
    """공통 논문 찾기 및 포맷팅"""
    if not papers_a or not papers_b:
        return "(실험을 먼저 실행하세요)", 0
    
    ids_a = {p.get('paperId') for p in papers_a if p.get('paperId')}
    ids_b = {p.get('paperId') for p in papers_b if p.get('paperId')}
    common_ids = ids_a & ids_b
    
    if not common_ids:
        return "(공통 논문 없음)", 0
    
    lines = [f"[{len(common_ids)}개 공통]"]
    for p in papers_a:
        if p.get('paperId') in common_ids:
            lines.append(f"• {p.get('title', 'N/A')[:50]}...")
    return "\n".join(lines), len(common_ids)

# 실험 A 데이터 수집
a_query = COMPARE_QUERY if 'COMPARE_QUERY' in dir() else ""
a_seeds = ""
a_results = ""
if 'result_1' in dir() and 'result_2' in dir():
    if result_1.get('seed') and result_2.get('seed'):
        a_seeds = f"시드1: {result_1['seed'].get('title', 'N/A')[:40]}...\n시드2: {result_2['seed'].get('title', 'N/A')[:40]}..."
    a_results = format_top10(result_1.get('selected', []))

# 실험 B 데이터 수집
b_seed = ""
b_queries = ""
b_results = ""
if 'query_result_1' in dir() and 'query_result_2' in dir():
    if query_result_1.get('seed'):
        b_seed = query_result_1['seed'].get('title', 'N/A')[:50] + "..."
    b_queries = f"쿼리1: {QUERY_1 if 'QUERY_1' in dir() else ''}\n쿼리2: {QUERY_2 if 'QUERY_2' in dir() else ''}"
    b_results = format_top10(query_result_1.get('selected', []))

# 공통 논문 (실험 A 기준)
common_text = "(실험을 먼저 실행하세요)"
if 'result_1' in dir() and 'result_2' in dir():
    common_text, _ = format_common_papers(
        result_1.get('selected', []), 
        result_2.get('selected', [])
    )

# ============================================================
# 입력 검증 및 제출 링크 생성
# ============================================================
if not PARTICIPANT_NAME:
    print("⚠️  PARTICIPANT_NAME을 입력하세요!")
elif a_results == "" and b_results == "":
    print("⚠️  실험 A 또는 B를 먼저 실행하세요!")
else:
    # Pre-filled Google Form URL 생성
    FORM_BASE = "https://docs.google.com/forms/d/e/1FAIpQLSfcDpL_hUwM0L_wzP3AwDiN0_EUTO4sqIZWTZ_ehVhQkJxkTg/viewform?usp=pp_url"
    
    params = {
        "entry.1560815365": PARTICIPANT_NAME,      # 이름
        "entry.1494848871": a_query,               # A_쿼리
        "entry.268560000": a_seeds,                # A_시드들
        "entry.1308825609": a_results,             # A결과_Top10
        "entry.604042982": b_seed,                 # B_시드
        "entry.767507188": b_queries,              # B_쿼리들
        "entry.1614784864": b_results,             # B결과_Top10
        "entry.1054173665": common_text,           # 공통논문
        # Q1~Q4는 비워두고 폼에서 직접 작성
    }
    
    form_url = FORM_BASE + "&" + urllib.parse.urlencode(params)
    
    # 결과 미리보기
    print("=" * 70)
    print("📋 제출 내용 미리보기")
    print("=" * 70)
    print(f"\n👤 이름: {PARTICIPANT_NAME}")
    print(f"\n🅰️ 실험 A")
    print(f"   쿼리: {a_query}")
    print(f"   시드: {a_seeds[:50]}...")
    print(f"\n🅱️ 실험 B")
    print(f"   시드: {b_seed[:50]}...")
    print(f"   쿼리: {b_queries}")
    print(f"\n📚 공통 논문: {common_text[:50]}...")
    
    # 클릭 가능한 링크 출력
    print("\n" + "=" * 70)
    print("🔗 아래 버튼 클릭 → 토론 질문 4개 작성 → 제출")
    print("=" * 70)
    
    display(HTML(f'''
    <a href="{form_url}" target="_blank" 
       style="display:inline-block; font-size:16px; padding:12px 24px; 
              background:#4285f4; color:white; text-decoration:none; 
              border-radius:5px; margin:10px 0;">
       📝 결과 제출하기 (클릭)
    </a>
    <p style="color:#666; font-size:12px;">※ 실험 결과는 자동 입력됩니다. 토론 질문만 작성하세요!</p>
    '''))
    
    # 로컬 백업 저장
    save_data = {
        'participant': PARTICIPANT_NAME,
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'experiment_A': {'query': a_query, 'seeds': a_seeds, 'results': a_results},
        'experiment_B': {'seed': b_seed, 'queries': b_queries, 'results': b_results},
        'common_papers': common_text
    }
    
    import json
    if IN_COLAB:
        save_folder = "/content/drive/MyDrive/aiworkshop_Feb2026/shared_results/"
    else:
        save_folder = os.path.join(WORKSHOP_DIR, "outputs", "comparison_results")
    
    os.makedirs(save_folder, exist_ok=True)
    filename = f"{PARTICIPANT_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    filepath = os.path.join(save_folder, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(save_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ 로컬 백업: {filepath}")

In [ ]:
# (강사용) 전체 참가자 결과 조회
# ============================================================
# Google Sheets에서 실시간으로 결과 읽기
# ============================================================

SHEET_URL = "https://docs.google.com/spreadsheets/d/15jyTrqGY7Po5iLcXFrv_kwyUNkCC9YMX6kypPMs-bAc/export?format=csv"

try:
    sheet_df = pd.read_csv(SHEET_URL)
    
    if len(sheet_df) > 0:
        print("=" * 70)
        print(f"📊 전체 참가자 결과 ({len(sheet_df)}명)")
        print("=" * 70)
        
        # 컬럼명 간소화
        new_cols = ['타임스탬프', '이름', 'A_쿼리', 'A_시드들', 'A결과', 
                    'B_시드', 'B_쿼리들', 'B결과', '공통논문',
                    'Q1_시드영향', 'Q2_쿼리영향', 'Q3_공통특징', 'Q4_적용전략']
        sheet_df.columns = new_cols[:len(sheet_df.columns)]
        
        # 요약 테이블 (주요 필드만)
        summary_cols = ['이름', 'A_쿼리', 'B_쿼리들', '공통논문']
        summary_df = sheet_df[[c for c in summary_cols if c in sheet_df.columns]].copy()
        
        # 긴 텍스트 자르기
        for col in summary_df.columns:
            if col != '이름':
                summary_df[col] = summary_df[col].apply(
                    lambda x: str(x)[:30] + "..." if pd.notna(x) and len(str(x)) > 30 else x
                )
        
        print("\n📋 요약 (쿼리 & 공통논문)")
        display(summary_df.style.hide(axis='index'))
        
        # 토론 응답 보기
        discussion_cols = ['이름', 'Q1_시드영향', 'Q2_쿼리영향', 'Q3_공통특징', 'Q4_적용전략']
        if all(c in sheet_df.columns for c in discussion_cols):
            print("\n" + "=" * 70)
            print("💬 토론 응답")
            print("=" * 70)
            
            for _, row in sheet_df.iterrows():
                print(f"\n👤 {row['이름']}")
                print(f"   Q1: {str(row.get('Q1_시드영향', ''))[:60]}...")
                print(f"   Q2: {str(row.get('Q2_쿼리영향', ''))[:60]}...")
                print(f"   Q3: {str(row.get('Q3_공통특징', ''))[:60]}...")
                print(f"   Q4: {str(row.get('Q4_적용전략', ''))[:60]}...")
        
        print(f"\n✓ 총 {len(sheet_df)}명 제출")
    else:
        print("⚠️  아직 제출된 결과가 없습니다.")
        
except Exception as e:
    print(f"⚠️  시트 읽기 오류: {e}")
    print("   Form 응답 시트가 연결되었는지 확인하세요.")
    print(f"\n💡 직접 확인: https://docs.google.com/spreadsheets/d/15jyTrqGY7Po5iLcXFrv_kwyUNkCC9YMX6kypPMs-bAc/")

---
# Part 5: DIY

본인의 연구 주제로 실험해보세요!

In [ ]:
# DIY - 본인의 연구 주제로 실험!
MY_DOI = None  # <- DOI 입력 (권장)
MY_TITLE = None  # <- DOI 없으면 제목으로 검색
MY_QUERY = None  # <- 검색 쿼리 (필수!)

# 입력 검증
if not MY_QUERY or MY_QUERY.strip() == "":
    print("⚠️  경고: MY_QUERY가 비어있습니다!")
    print("   검색에 사용할 쿼리를 입력하세요.")
    print("   예: MY_QUERY = \"depression detection machine learning\"")
    my_result = {'selected': []}
elif not MY_DOI and not MY_TITLE:
    print("⚠️  경고: 시드 정보가 없습니다!")
    print("   MY_DOI 또는 MY_TITLE 중 하나를 입력하세요.")
    print("   예: MY_DOI = \"10.1000/your.paper.doi\"")
    print("   예: MY_TITLE = \"Your paper title here\"")
    my_result = {'selected': []}
else:
    my_result = run_pipeline(
        seed_doi=MY_DOI, 
        seed_title=MY_TITLE if not MY_DOI else None, 
        query=MY_QUERY
    )
    
    if my_result['selected']:
        print("\n" + "=" * 60)
        print("내 시드 기반 Top 10")
        print("=" * 60)
        for i, p in enumerate(my_result['selected'], 1):
            print(f"{i}. [{p.get('relevance_score', 0):.3f}] {p.get('title', 'N/A')[:50]}...")

---
## 결론

### 배운 것
| 도구 | 역할 | 링크 |
|------|------|------|
| **paperscraper** | 키워드로 시드 후보 검색 | [GitHub](https://github.com/jannisborn/paperscraper) |
| **Semantic Scholar** | 인용/참조/추천 데이터 | [API Docs](https://api.semanticscholar.org/api-docs/) |
| **SPECTER2** | 논문 임베딩 → 관련성 점수 | [GitHub](https://github.com/allenai/SPECTER2) \| [논문](https://arxiv.org/abs/2211.13308) |

### 핵심 인사이트
- **Crawler**: 양적 확장 (References + Citations + Related)
- **Selector**: 질적 필터링 (쿼리-논문 유사도)
- **시드 선택이 중요**: 출발점에 따라 결과가 달라짐

### 이 접근법의 한계
- paperscraper: arXiv/PubMed만 검색 (Google Scholar 커버리지보다 좁음)
- Crawler: 1-hop만 확장 (깊이 제한)
- Selector: 초록 없는 논문은 점수가 낮아짐

### 추가 학습 자료
- [PaSa: An LLM Agent for Comprehensive Academic Paper Search](https://arxiv.org/abs/2501.10120) - 이 노트북의 영감이 된 논문
- [OpenAlex](https://openalex.org/) - 완전 무료 학술 데이터베이스 API
- [Semantic Scholar Open Research Corpus](https://allenai.org/data/s2orc) - 대규모 논문 데이터셋